In [ ]:
import purly

In [ ]:
from example_utils import localhost
purly.state.Machine().daemon()

# name the layout resource "color-wheel" and connect to the update stream
websocket_url = localhost('ws', 8000) + '/model/selection/stream'

In [ ]:
layout = purly.Layout(websocket_url)

In [ ]:
def grid(x, y, size):

    container = layout.html('div', style={"display": "table"})
    state = {"dragging": False, "selection": set(), "start": (0, 0), "stop": (0, 0)}
    
    def square(size, color, margin):
        return layout.html(
            'div',
            style={
                "display": "table-cell",
                "height": "%spx" % size,
                "width": "%spx" % size,
                "backgroundColor": "blue",
                "border": "2px solid white"
            },
        )
    
    def select():
        new = set()
        x1, y1 = state["start"]
        x2, y2 = state["stop"]
        start_x, stop_x = sorted([x1, x2])
        start_y, stop_y = sorted([y1, y2])
        for i in range(start_x, stop_x + 1):
            for j in range(start_y, stop_y + 1):
                new.add((i, j))
        for (x, y) in state["selection"].difference(new):
            container.children[x].children[y].style["backgroundColor"] = "blue"
        for (x, y) in new:
            container.children[x].children[y].style["backgroundColor"] = "red"
        state["selection"].update(new)

    def clear():
        for (x, y) in state["selection"]:
            container.children[x].children[y].style["backgroundColor"] = "blue"
        state["selection"].clear()

    for i in range(y):
        row = layout.html('div', style={"display": "table-row"})
        for j in range(x):
            sqr = square(size, 'blue', size)

            @sqr.on("MouseDown")
            def drag_start(_x=i, _y=j):
                state["stop"] = state["start"] = (_x, _y)
                state["dragging"] = True
                select()

            @sqr.on("MouseUp")
            def drag_stop():
                if state["stop"]:
                    state["dragging"] = False
                    clear()

            @sqr.on("MouseEnter")
            def drag_select(_x=i, _y=j):
                if state["dragging"]:
                    if state["stop"] != (_x, _y):
                        state["stop"] = (_x, _y)
                        select()

            row.children.append(sqr)
        container.children.append(row)

    return container

In [ ]:
g = grid(30, 10, 30)

In [ ]:
layout.children.clear()
layout.sync()

In [ ]:
layout.children.append(g)
layout.sync()

In [ ]:
layout

In [ ]:
layout.serve()